In [ ]:
import keras_nlp as keras
import pandas as pd
import numpy as np
import tensorflow as tf
path = './second_dataset/mbti_1.csv' 
#path =  './third_dataset/MBTI 500.csv'
df = pd.read_csv(path)
train_fraction = 0.80
val_fraction = 0.10
vocab_size = 20000
seq_len = 256
batch_size = 64
vocab = "vocab"
seed = 0
preset_token = ["__url__", "__sep__", "__tar__"]

In [6]:
def preprocess_df(df):
    posts = df["posts"]
    types = df["type"]
    
    posts = posts.astype(str).str.replace(r'http[^\s|]+', ' __url__ ', regex=True)
    posts = posts.astype(str).str.replace(r'|||', ' __sep__ ', regex=False)
    posts = posts.astype(str).str.replace(r'(?i)\b[IE][NS][TF][JP]\b', ' __tar__ ', regex=True) # i.e. INTP, ENFJ
    posts = posts.astype(str).str.replace(r"\s+", " ", regex=True) # remove excess spaces
    
    types = types.astype(str).str.replace(r'I|N|T|J', "1", regex=True)
    types = types.astype(str).str.replace(r'E|S|F|P', "0", regex=True)
    
    df["posts"] = posts
    df["type"] = types
    
    df[["I_E", "N_S", "T_F", "J_P"]] = (
        df["type"].apply(lambda s: pd.Series([int(c) for c in s]))
    ).astype("int32")

    return df

In [7]:
# train/test
df = preprocess_df(df)
print(df)
n = len(df)
n_train = int(train_fraction * n)
n_val = int((val_fraction + train_fraction) * n)

train_df = df[ : n_train]
val_df = df[n_train : n_val]
test_df = df[n_val : ]

      type                                              posts  I_E  N_S  T_F  \
0     1101  ' __url__ __sep__ __url__ __sep__ __tar__ and ...    1    1    0   
1     0110  'I'm finding the lack of me in these posts ver...    0    1    1   
2     1110  'Good one _____ __url__ __sep__ Of course, to ...    1    1    1   
3     1111  'Dear __tar__ , I enjoyed our conversation the...    1    1    1   
4     0111  'You're fired. __sep__ That's another silly mi...    0    1    1   
...    ...                                                ...  ...  ...  ...   
8670  1000  ' __url__ __sep__ IxFP just because I always t...    1    0    0   
8671  0100  'So...if this thread already exists someplace ...    0    1    0   
8672  1110  'So many questions when i do these things. I w...    1    1    1   
8673  1100  'I am very conflicted right now when it comes ...    1    1    0   
8674  1100  'It has been too long since I have been on per...    1    1    0   

      J_P  
0       1  
1       0  
2  

In [8]:

def create_tf_ds(posts, targets, training=False):
    posts = posts.astype(str).to_numpy()
    targets = targets.to_numpy(dtype="float32")
    
    ds = tf.data.Dataset.from_tensor_slices((posts, targets))
    if training:
        ds = ds.shuffle(buffer_size=len(train_df), seed=seed, reshuffle_each_iteration=True)
    ds = ds.batch(batch_size)
    return ds

train_ds = create_tf_ds(train_df["posts"], train_df[["I_E", "N_S", "T_F", "J_P"]], training=True)
val_ds   = create_tf_ds(val_df["posts"],   val_df[["I_E", "N_S", "T_F", "J_P"]], training=False)
test_ds  = create_tf_ds(test_df["posts"],  test_df[["I_E", "N_S", "T_F", "J_P"]], training=False)

In [ ]:
from tokenizers import Tokenizer
from tokenizers.models import WordPiece
from tokenizers.trainers import WordPieceTrainer
from tokenizers.normalizers import BertNormalizer
from tokenizers.pre_tokenizers import BertPreTokenizer
from pathlib import Path

train_texts = train_df["posts"].astype(str).tolist()


tok = Tokenizer(WordPiece(unk_token="__unk__"))
tok.normalizer = BertNormalizer(lowercase=True)
tok.pre_tokenizer = BertPreTokenizer()

trainer = WordPieceTrainer(vocab_size=vocab_size, special_tokens=preset_token)
tok.train_from_iterator(train_texts, trainer=trainer)

tok.save(path + "/../tokenizer.json")
print("saved at:", Path(path + "/../tokenizer.json").resolve())


saved in: C:\Users\jeldr\ML-Project\second_dataset\tokenizer.json
